In [23]:
# ML II Assigment - Alan Corrales - Gracias por todo Conchita!


In [31]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# Load the dataset
file_path = 'US_Regional_Sales_Data.csv'  
data = pd.read_csv(file_path)
US_Sales = data
data.head()

,OrderNumber,Sales Channel,WarehouseCode,ProcuredDate,OrderDate,ShipDate,DeliveryDate,CurrencyCode,_SalesTeamID,_CustomerID,_StoreID,_ProductID,Order Quantity,Discount Applied,Unit Cost,Unit Price,Total Sales
0,SO - 000101,In-Store,WARE-UHY1004,31/12/2017,31/05/2018,14/06/2018,19/06/2018,USD,6,15,259,12,5,0.075,"1,001.18","1,963.10",9815.5
1,SO - 000102,Online,WARE-NMK1003,31/12/2017,31/05/2018,22/06/2018,02/07/2018,USD,14,20,196,27,3,0.075,"3,348.66","3,939.60",11818.8
2,SO - 000103,Distributor,WARE-UHY1004,31/12/2017,31/05/2018,21/06/2018,01/07/2018,USD,21,16,213,16,1,0.050,781.22,"1,775.50",1775.5
3,SO - 000104,Wholesale,WARE-NMK1003,31/12/2017,31/05/2018,02/06/2018,07/06/2018,USD,28,48,107,23,8,0.075,"1,464.69","2,324.90",18599.2
4,SO - 000105,Distributor,WARE-NMK1003,10/04/2018,31/05/2018,16/06/2018,26/06/2018,USD,22,49,111,26,8,0.100,"1,476.14","1,822.40",14579.2


In [32]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7991 entries, 0 to 7990
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   OrderNumber       7991 non-null   object 
 1   Sales Channel     7991 non-null   object 
 2   WarehouseCode     7991 non-null   object 
 3   ProcuredDate      7991 non-null   object 
 4   OrderDate         7991 non-null   object 
 5   ShipDate          7991 non-null   object 
 6   DeliveryDate      7991 non-null   object 
 7   CurrencyCode      7991 non-null   object 
 8   _SalesTeamID      7991 non-null   int64  
 9   _CustomerID       7991 non-null   int64  
 10  _StoreID          7991 non-null   int64  
 11  _ProductID        7991 non-null   int64  
 12  Order Quantity    7991 non-null   int64  
 13  Discount Applied  7991 non-null   float64
 14  Unit Cost         7991 non-null   object 
 15  Unit Price        7991 non-null   object 
 16  Total Sales       10 non-null     float64


In [33]:
#RandomForest Model


# Function to convert currency to float
def currency_to_float(currency_val):
    if isinstance(currency_val, str):
        return float(currency_val.replace(',', '').replace('$', ''))
    return currency_val

# Convert currency fields from string to numeric
currency_columns = ['Unit Cost', 'Unit Price', 'Total Sales']
for col in currency_columns:
    data[col] = data[col].apply(currency_to_float)

# Convert date fields to datetime
date_columns = ['ProcuredDate', 'OrderDate', 'ShipDate', 'DeliveryDate']
for col in date_columns:
    data[col] = pd.to_datetime(data[col], errors='coerce')

# Feature Engineering - Example: Calculate the delivery lead time
data['DeliveryLeadTime'] = (data['DeliveryDate'] - data['OrderDate']).dt.days

# Drop the original date columns and any other non-relevant columns
data.drop(columns=date_columns, inplace=True)

# Defining the target variable
target = 'Total Sales'

# Drop any rows with missing target values
data.dropna(subset=[target], inplace=True)

# Separating features and target
X = data.drop(columns=[target])
y = data[target]

# Identifying categorical columns (after removing date columns)
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()

# Creating a ColumnTransformer for transformations
preprocessor = ColumnTransformer(
    transformers=[
        ('num', SimpleImputer(strategy='median'), X.select_dtypes(include=['int64', 'float64']).columns),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ])

# Create the preprocessing and modeling pipeline for random forest
rf_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', RandomForestRegressor(n_estimators=100, random_state=42))])

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit the random forest pipeline
rf_pipeline.fit(X_train, y_train)

# Predict and evaluate the random forest model
y_pred_rf = rf_pipeline.predict(X_test)
mse_rf = mean_squared_error(y_test, y_pred_rf)

# Output the MSE
print(f"Random Forest MSE: {mse_rf}")


C:\Users\alanc\AppData\Local\Temp\ipykernel_22500\4004328527.py:18: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  data[col] = pd.to_datetime(data[col], errors='coerce')
C:\Users\alanc\AppData\Local\Temp\ipykernel_22500\4004328527.py:18: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  data[col] = pd.to_datetime(data[col], errors='coerce')
C:\Users\alanc\AppData\Local\Temp\ipykernel_22500\4004328527.py:18: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  data[col] = pd.to_datetime(data[col], errors='coerce')
C:\Users\alanc\AppData\Local\Temp\ipykernel_22500\40043

Random Forest MSE: 17212122.42320002


In [37]:
import pickle

with open('RFbest_model.pkl', 'wb') as file:
    pickle.dump(rf_pipeline, file)

In [38]:
# Linear Regression Model

In [39]:
# Load the dataset
file_path = 'C:/Users/alanc/Downloads/US_Regional_Sales_Data.csv'  
data = pd.read_csv(file_path)
US_Sales = data
data.head()

,OrderNumber,Sales Channel,WarehouseCode,ProcuredDate,OrderDate,ShipDate,DeliveryDate,CurrencyCode,_SalesTeamID,_CustomerID,_StoreID,_ProductID,Order Quantity,Discount Applied,Unit Cost,Unit Price,Total Sales
0,SO - 000101,In-Store,WARE-UHY1004,31/12/2017,31/05/2018,14/06/2018,19/06/2018,USD,6,15,259,12,5,0.075,"1,001.18","1,963.10",9815.5
1,SO - 000102,Online,WARE-NMK1003,31/12/2017,31/05/2018,22/06/2018,02/07/2018,USD,14,20,196,27,3,0.075,"3,348.66","3,939.60",11818.8
2,SO - 000103,Distributor,WARE-UHY1004,31/12/2017,31/05/2018,21/06/2018,01/07/2018,USD,21,16,213,16,1,0.050,781.22,"1,775.50",1775.5
3,SO - 000104,Wholesale,WARE-NMK1003,31/12/2017,31/05/2018,02/06/2018,07/06/2018,USD,28,48,107,23,8,0.075,"1,464.69","2,324.90",18599.2
4,SO - 000105,Distributor,WARE-NMK1003,10/04/2018,31/05/2018,16/06/2018,26/06/2018,USD,22,49,111,26,8,0.100,"1,476.14","1,822.40",14579.2


In [40]:


# Function to convert currency to float
def currency_to_float(currency_val):
    if isinstance(currency_val, str):
        return float(currency_val.replace(',', '').replace('$', ''))
    return currency_val

# Convert currency fields from string to numeric
currency_columns = ['Unit Cost', 'Unit Price', 'Total Sales']
for col in currency_columns:
    data[col] = data[col].apply(currency_to_float)

# Convert date fields to datetime
date_columns = ['ProcuredDate', 'OrderDate', 'ShipDate', 'DeliveryDate']
for col in date_columns:
    data[col] = pd.to_datetime(data[col], errors='coerce')

# Feature Engineering - Example: Calculate the delivery lead time
data['DeliveryLeadTime'] = (data['DeliveryDate'] - data['OrderDate']).dt.days

# Drop the original date columns and any other non-relevant columns
data.drop(columns=date_columns, inplace=True)

# Defining the target variable
target = 'Total Sales'

# Drop any rows with missing target values
data.dropna(subset=[target], inplace=True)

# Separating features and target
X = data.drop(columns=[target])
y = data[target]

# Identifying categorical columns (after removing date columns)
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()

# Creating a ColumnTransformer for transformations
preprocessor = ColumnTransformer(
    transformers=[
        ('num', SimpleImputer(strategy='median'), X.select_dtypes(include=['int64', 'float64']).columns),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ])

# Create the preprocessing and modeling pipeline for linear regression
lr_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', LinearRegression())])

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit the linear regression pipeline
lr_pipeline.fit(X_train, y_train)

# Predict and evaluate the linear regression model
y_pred_lr = lr_pipeline.predict(X_test)
mse_lr = mean_squared_error(y_test, y_pred_lr)

# Output the MSE
print(f"Linear Regression MSE: {mse_lr}")


Linear Regression MSE: 27575094.976452537


C:\Users\alanc\AppData\Local\Temp\ipykernel_22500\3295647457.py:15: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  data[col] = pd.to_datetime(data[col], errors='coerce')
C:\Users\alanc\AppData\Local\Temp\ipykernel_22500\3295647457.py:15: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  data[col] = pd.to_datetime(data[col], errors='coerce')
C:\Users\alanc\AppData\Local\Temp\ipykernel_22500\3295647457.py:15: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  data[col] = pd.to_datetime(data[col], errors='coerce')
C:\Users\alanc\AppData\Local\Temp\ipykernel_22500\32956

In [41]:

import pickle

with open('LRbest_model.pkl', 'wb') as file:
    pickle.dump(lr_pipeline, file)